In [1]:
import pandas as pd
from glob import glob
import numpy as np
from aggmap import AggMap, AggModel

# small smaple size data tends to generate random fmaps

In [2]:
flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()

In [3]:
fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)

In [4]:
df = pd.concat(fall, axis=0)

In [5]:
df = df.sample(317, random_state = 123)

In [6]:
df.groupby('class').size().to_frame(name = 'count').to_csv('classes.csv')

In [7]:
dfx = df[df.columns[:-1]]
dfy = df[df.columns[-1:]]

In [8]:
dfx = np.log2(dfx + 1) #apply log2(x + 1)

In [9]:
mp = AggMap(dfx)

2021-03-16 13:15:35,678 - INFO - [bidd-aggmap] - Calculating distance ...
2021-03-16 13:15:43,374 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 10381/10381 [00:17<00:00, 601.58it/s]


In [10]:
mp.fit(cluster_channels = 5, verbose = 0)

2021-03-16 13:51:39,833 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2021-03-16 13:52:09,493 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2021-03-16 14:01:57,004 - INFO - [bidd-aggmap] - Finished


In [11]:
mp.plot_scatter(htmlname = 'relatively-low-sample')

2021-03-16 14:01:57,074 - INFO - [bidd-aggmap] - generate file: ./relatively-low-sample_feature points_10381_correlation_umap_scatter
2021-03-16 14:01:57,246 - INFO - [bidd-aggmap] - save html file to ./relatively-low-sample_feature points_10381_correlation_umap_scatter


In [12]:
mp.plot_grid(htmlname = 'relatively-low-sample')

2021-03-16 14:01:57,410 - INFO - [bidd-aggmap] - generate file: ./relatively-low-sample_feature points_10381_correlation_umap_mp
2021-03-16 14:01:57,567 - INFO - [bidd-aggmap] - save html file to ./relatively-low-sample_feature points_10381_correlation_umap_mp


In [35]:
import os
from aggmap import loadmap
from joblib import dump,load

data_save_folder = '/raid/shenwanxiang/transcriptome/pan-cancer/relative-low-sample'

flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()
fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)
df = pd.concat(fall, axis=0)
dfx = df[df.columns[:-1]]
dfx = np.log2(dfx + 1) #apply log2(x + 1)

scale_method = 'minmax'
mp_larger = loadmap('/raid/shenwanxiang/agg_mp_object/pan-cancer.mp')
mp.x_min = mp_larger.x_min
mp.x_max = mp_larger.x_max



if not os.path.exists(data_save_folder):
    os.makedirs(data_save_folder)
    
if not os.path.exists(os.path.join(data_save_folder, 'Agg5.data')):
    # save multi-channel
    X = mp.batch_transform(dfx.values, scale_method = scale_method) 
    n, w, h, c = X.shape
    dump(X,  os.path.join(data_save_folder, 'Agg5.data')) 
    
    ## save single channel
    X1 = X.sum(axis=-1).reshape(n, w, h, 1)
    dump(X1,  os.path.join(data_save_folder, 'Agg1.data')) 

else:
    X = load(os.path.join(data_save_folder, 'Agg5.data'))
    
    
mp.save(os.path.join(data_save_folder, 'relative_low.mp'))

100%|##########| 10446/10446 [32:15<00:00,  5.40it/s]


['/raid/shenwanxiang/transcriptome/pan-cancer/relative-low-sample/relative_low.mp']